In [3]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [59]:
import requests
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.render import render_text_description

class LockColor(BaseModel):
    color: str = Field(description="first color")

@tool
def sendRequest(color: str):
    """Sends the color to another place"""
    response = requests.post("http://localhost:3000/api/lockcolor", json={"body": {"color": color}})
    print("IM INSIDE THIS BIT")
    try:
        print(response)
    except Exception as e:
        print('error: ', e)

    return f"went through with {color}"

req = StructuredTool.from_function(
    func=sendRequest,
    name="SendRequest",
    description="Send a request to localhost",
    args_schema=LockColor,
    handle_tool_error=True,
)

In [55]:
sendRequest.invoke('blue')
req.run('blue')

IM INSIDE THIS BIT
error
IM INSIDE THIS BIT
error


'went through with blue'

In [60]:
rendered_tools = render_text_description([sendRequest])
rendered_tools

'sendRequest: sendRequest(color: str) - Sends the color to another place'

In [61]:
model = ChatOpenAI(model="gpt-3.5-turbo")

model_with_function = model.bind_tools([req])


# prompt = ChatPromptTemplate.from_template("send a request to localhost with the color {color}. Return the response of the function and NOTHING ELSE.")

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.

You have a great eye for color.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)


# output_parser = StrOutputParser()
output_parser = JsonOutputParser()

chain = prompt | model | output_parser

In [70]:
chain.invoke({"input": "Send in the blue squadran"})

IM INSIDE THIS BIT
<Response [200]>


'went through with blue squadran'

In [ ]:
from operator import itemgetter

chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | sendRequest
chain.invoke({"input": "whats blue and round?"})

In [71]:
requests.post("http://localhost:3000/api/lockcolor", data={"key": "value"},
headers={"Content-Type": "application/json"},
)

<Response [200]>